In [ ]:
import os
from datetime import datetime,timedelta
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
cmorph_prd = '/cpc/fews/production/cmorph_cpcwork5/cmorph_RH6_BC_ADJ_EOD/output/bin/CMORPH_V1.0_ADJ_0.25deg-DLY_EOD_'
#from Ravi's ctl file for cmorph
xdim = 1440
ydim = 480
xmin = 0.125
xmax = 360
ymin = -59.875
ymax = 60

In [ ]:
def get_date_str(date):
    if date.month <10:
        cmonth_str = '0' + str(date.month)
    else:
        cmonth_str = str(date.month)
    if date.day <10:
        cday_str = '0' + str(date.day)
    else: cday_str = str(date.day)
    current_str = str(date.year) + cmonth_str + cday_str
    return current_str

In [ ]:
def read_in_binary(location, xdim, ydim, xmin, xmax, ymin, ymax):
    with open(location, 'rb') as binary_file:
        binary_data = binary_file.read()
    data = np.frombuffer(binary_data, dtype = np.float32)
    data = data.reshape(ydim, xdim)
    lat = np.linspace(ymin, ymax, data.shape[0])
    lon = np.linspace(xmin, xmax, data.shape[1])
    da = xr.DataArray(data, dims = ['lat', 'lon'], coords = {'lat': lat, 'lon':lon})
    return da

In [ ]:
from datetime import datetime, timedelta

# Get the current date
current_date = datetime.now().date()

# Get the date 7 days prior
seven_days_ago = current_date - timedelta(days=8)

# Generate a list of dates from seven_days_ago to current_date
week1 = [seven_days_ago + timedelta(days=i) for i in range(7)]  # 8 to include both endpoints

date_strs = []
for date in week1:
    date_strs.append(get_date_str(date))

In [ ]:
week1_da = []
for d, date in enumerate(date_strs):
    da = read_in_binary(cmorph_prd + date, xdim, ydim, xmin, xmax, ymin, ymax)
    da = da.expand_dims({'date': [week1[d]]})
    week1_da.append(da)

week1_ds = xr.concat(week1_da, dim = 'date').sum(dim = 'date').to_dataset(name = 'weekly_cmorph')
plt.figure()
quadmesh = week1_ds.weekly_cmorph.plot()
ax = plt.gca()
ax.axis('off')
plt.savefig('weeklycmorph.png', bbox_inches = 'tight', pad_inches = 0)